In [ ]:
! pip install torch

python: can't open file '/Users/nishidayuutarou/dev/indv/ml/train.py': [Errno 2] No such file or directory


<h2>予測コード</h2>


In [ ]:
import torch
import torch.nn as nn

class SimpleGPTPredictor(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # K, V, Qとかの計算->残渣結合->正規化->FFN (Feed Forward Network)とかをやってるぽい
        # Attention計算では、QとKの（各トークンのKにたいする）内積を計算して、それをベクトルとしてもつ。そのベクトルをSoftmaxで重みにする。
        # 各トークンのVと重みつき平均を取る。
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embed_size, num_heads),
            num_layers=2
        )

        self.lm_head = nn.Linear(embed_size, vocab_size)

    def forward(self, x):
        # 埋め込み
        embedded = self.embedding(x)
        # 文脈情報を考慮してベクトルを更新（形状そのまま）
        transformed = self.transformer(embedded)
        # 非正規化前のスコア
        output = self.lm_head(transformed)
        
        return output

with open('inputLearnText.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = list(set(text))  #hello->heloみたいにする。
char_to_id = {ch: i for i, ch in enumerate(chars)} # 文字: idの配列
id_to_char = {i: ch for i, ch in enumerate(chars)} # id: 文字の配列

print("文字→ID辞書:", char_to_id)

# テキストを1文字ずつIDに変換
def text_to_ids(text):
    return [char_to_id[ch] for ch in text]

# IDを1文字ずつテキストに変換
def ids_to_text(ids):
    return ''.join([id_to_char[i] for i in ids])



文字→ID辞書: {'”': 0, '0': 1, 'k': 2, 'ה': 3, 'μ': 4, 'ἀ': 5, 'y': 6, '-': 7, 'υ': 8, 'ε': 9, 'j': 10, 'χ': 11, 'm': 12, 'O': 13, 'E': 14, 'Y': 15, '“': 16, '.': 17, 'C': 18, ',': 19, 'W': 20, 'P': 21, 'M': 22, 'ύ': 23, 'F': 24, '9': 25, 'G': 26, ' ': 27, '7': 28, '\n': 29, 'ְ': 30, 'h': 31, 'ι': 32, 'ח': 33, 'f': 34, 'v': 35, ']': 36, 'b': 37, 'י': 38, 'i': 39, 'מ': 40, '1': 41, 'g': 42, 'ρ': 43, ':': 44, 'p': 45, 'ἐ': 46, '’': 47, 'ו': 48, 'o': 49, 'ή': 50, 'η': 51, '?': 52, 'σ': 53, 'ν': 54, 'ῃ': 55, 'n': 56, 'e': 57, 'V': 58, 'U': 59, 'X': 60, 'J': 61, 'ῆ': 62, 'L': 63, '6': 64, '—': 65, 'ί': 66, 'α': 67, 'כ': 68, 'ς': 69, 'ῥ': 70, 'ῳ': 71, ')': 72, 'τ': 73, 'ο': 74, 'ὕ': 75, 'z': 76, '8': 77, 'S': 78, 'l': 79, 'w': 80, 'κ': 81, 'd': 82, 'H': 83, 'D': 84, '[': 85, 'Æ': 86, 'u': 87, '2': 88, '5': 89, 'π': 90, 'R': 91, 'x': 92, 'ō': 93, 'ῤ': 94, 'A': 95, 'B': 96, 'æ': 97, 'ό': 98, 'c': 99, 'ἰ': 100, 't': 101, 'T': 102, 'ָ': 103, '(': 104, 'ώ': 105, 'ᾶ': 106, 'q': 107, '4': 108, 'ω': 109,

<h2>学習用コード</h2>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# テキストを1文字ずつIDに変換
def text_to_ids(text):
    return [char_to_id[ch] for ch in text]

# IDを1文字ずつテキストに変換
def ids_to_text(ids):
    return ''.join([id_to_char[i] for i in ids])

# 簡単な学習データ作成
def create_training_data(text, seq_len=10):
    ids = text_to_ids(text)
    inputs, targets = [], []

    for i in range(len(ids) - seq_len):
        inputs.append(ids[i:i+seq_len])        # 入力：10文字
        targets.append(ids[i+1:i+seq_len+1])   # 正解：1文字ずらし

    return torch.tensor(inputs), torch.tensor(targets)

# 学習データ作成
# 追加する
with open('inputLearnText.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# これなんだっけ
chars = set(text)

char_to_id = {ch: i for i, ch in enumerate(chars)} # 文字: idの配列
id_to_char = {i: ch for i, ch in enumerate(chars)} # id: 文字の配列

train_inputs, train_targets = create_training_data(text)

print(f"学習データ数: {len(train_inputs)}")
print(f"例 - 入力: '{ids_to_text(train_inputs[20].tolist())}'")
print(f"例 - 正解: '{ids_to_text(train_targets[20].tolist())}'")


model = SimpleGPTPredictor(vocab_size=len(chars), embed_size=32, num_heads=4)
# 学習設定
optimizer = optim.Adam(model.parameters(), lr=0.001)
# 損失関数の種類
criterion = nn.CrossEntropyLoss()

print("\n学習開始...")
# range
train_inputs = train_inputs[:5000]
train_targets = train_targets[:5000]

for epoch in range(100):
    total_loss = 0
    batchSize = 128
    for i in range(0, len(train_inputs), batchSize):
        optimizer.zero_grad()

        # 予測 この記法でforward()を実行できる。
        # 出力は正規化前の値 例[2.1, -0.5, 3.2, 0.8, -1.1, ...]
        output = model(train_inputs[i:i+batchSize])
        # これもcall?
        loss = criterion(output.view(-1, len(chars)), train_targets[i:i+batchSize].view(-1))

        # 学習
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss/len(train_inputs):.4f}")

print("学習完了！")

学習データ数: 51642
例 - 入力: ' Fathers w'
例 - 正解: 'Fathers wi'

学習開始...


<h2>予測テスト</h2>

In [61]:
def test_prediction(model, input_text):
    input_ids = text_to_ids(input_text)
    input_tensor = torch.tensor([input_ids])

    with torch.no_grad():
        output = model(input_tensor)
        last_char_probs = output[0, -1, :]
        probs = torch.softmax(last_char_probs, dim=-1)

        # 一位のトークンを呼び出す。
        top_prob, top_index = torch.topk(probs, 1)
        char_id = top_index.item()  # テンソルから数値を取り出し
        predicted_char = id_to_char[char_id]  # IDを文字に変換

        return predicted_char

def generateSeq(model, text, count = 0):
    nextSingleToken = test_prediction(model, text)
    if(count < 20):
        return generateSeq(model, text+nextSingleToken, count+1)
    else:
        return text+nextSingleToken
    
prompt = "おはよう"
completion = generateSeq(model, prompt)
print("入力テキスト: ", prompt)
print("回答: ", completion)

KeyError: 'お'